# Computer Vision And Image Analytics

## Assignment Description Object Detection using YOLOv8 and Faster R-CNN

Objective:

Implement and compare YOLOv8 and Faster R-CNN for wheat head detection in terms of accuracy, speed, and robustness.


In [21]:
# STEP 1 — Environment setup (Colab)
# 1) Check GPU
!nvidia-smi || echo "No GPU found. In Colab, set: Runtime > Change runtime type > GPU"

# 2) Install key libs
# - ultralytics (YOLOv8)
# - torchmetrics (for mAP on Faster R-CNN)
# - pycocotools (COCO metrics support)
# - opencv-python (image I/O)
!pip -q install ultralytics==8.3.0 torchmetrics==1.4.1 pycocotools opencv-python

# 3) Quick sanity imports (should print versions, not error)
import torch, torchvision, cv2, ultralytics, torchmetrics
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("OpenCV:", cv2.__version__)
print("ultralytics:", ultralytics.__version__)
print("torchmetrics:", torchmetrics.__version__)


Sat Sep 20 07:11:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             30W /   70W |     242MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!file "/content/global-wheat-detection.zip"
!ls -lh "/content/global-wheat-detection.zip"


/content/global-wheat-detection.zip: Zip archive data, at least v4.5 to extract, compression method=deflate
-rw-r--r-- 1 root root 19M Sep 20 10:02 /content/global-wheat-detection.zip


In [2]:
!kaggle competitions download -c global-wheat-detection -p /content/
!unzip -q /content/global-wheat-detection.zip -d /content/global-wheat-detection


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
[/content/global-wheat-detection.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/global-wheat-detection.zip or
        /content/global-wheat-det

In [3]:
!unzip -q "/content/train.zip" -d /content/global-wheat-detection


unzip:  cannot find or open /content/train.zip, /content/train.zip.zip or /content/train.zip.ZIP.


In [6]:
!unzip -q "/content/train.zip" -d /content/global-wheat-detection.zip


[/content/train.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/train.zip or
        /content/train.zip.zip, and cannot find /content/train.zip.ZIP, period.


In [8]:
# STEP 2c — Verify & extract archives robustly (tries global-wheat-detection.zip first)

import os, subprocess, glob, shutil
from pathlib import Path

BASE = Path("/content")
TARGET = Path("/content/global-wheat-detection")
TARGET.mkdir(parents=True, exist_ok=True)

def sh(cmd):
    rc = subprocess.call(cmd, shell=True)
    return rc

def zip_ok(p: Path) -> bool:
    # Test archive integrity
    return sh(f'unzip -t "{p}" > /dev/null') == 0

print("📂 Looking for dataset archives in /content ...")
candidates = []
for name in ["global-wheat-detection.zip", "train.zip"]:
    p = BASE / name
    if p.exists():
        candidates.append(p)

print("Found:", [c.name for c in candidates])

# Try global-wheat-detection.zip first (contains CSV + train/)
archive = None
for cand in candidates:
    print(f"🔎 Testing {cand.name} ...")
    if zip_ok(cand):
        archive = cand
        print(f"✅ Zip integrity OK: {cand.name}")
        break
    else:
        print(f"❌ Zip test failed: {cand.name}")

if archive is None:
    raise RuntimeError(
        "No valid zip archive found. Please re-upload 'global-wheat-detection.zip' "
        "(~1.6GB) OR re-upload a good 'train.zip'."
    )

print(f"\n📦 Extracting {archive.name} ... this may take a minute ...")
rc = sh(f'unzip -oq "{archive}" -d "{TARGET}"')
if rc != 0:
    raise RuntimeError(f"Unzip failed with exit code {rc} for {archive.name}")

# Sanity check structure
csv_path = TARGET / "train.csv"
train_dir = TARGET / "train"

# Some zips may create nested directory; try to locate if not at root
if not train_dir.exists():
    # Search for a 'train' folder inside
    nested = [p for p in TARGET.rglob("train") if p.is_dir()]
    if nested:
        # Pick the one with most jpgs
        best = None; bestn = -1
        for d in nested:
            n = len(list(d.glob("*.jpg")))
            if n > bestn:
                best, bestn = d, n
        if best and best != train_dir:
            train_dir.mkdir(exist_ok=True, parents=True)
            print(f"Normalizing: copying train images from {best} -> {train_dir}")
            shutil.copytree(best, train_dir, dirs_exist_ok=True)

# Count images
jpgs = list(train_dir.glob("*.jpg")) if train_dir.exists() else []
print(f"\n🖼️ Images found in {train_dir}: {len(jpgs)}")

# CSV: ensure present (you may have uploaded it earlier separately)
if not csv_path.exists():
    # Try to find a train.csv somewhere under TARGET and move it up
    candidates_csv = list(TARGET.rglob("train.csv"))
    if candidates_csv:
        print(f"🔧 Moving CSV from {candidates_csv[0]} to {csv_path}")
        shutil.copy2(candidates_csv[0], csv_path)

# Final checks
problems = []
if not train_dir.exists() or len(jpgs) == 0:
    problems.append("train images missing")

if not csv_path.exists():
    problems.append("train.csv missing")

if problems:
    raise RuntimeError("❌ Setup incomplete: " + ", ".join(problems) +
                       "\nPlease re-upload a valid archive or use Kaggle API.")

print("\n✅ Ready: dataset is prepared at /content/global-wheat-detection")
print("   - CSV:", csv_path.exists(), "| path:", csv_path)
print("   - Train images:", len(jpgs))


📂 Looking for dataset archives in /content ...
Found: ['global-wheat-detection.zip', 'train.zip']
🔎 Testing global-wheat-detection.zip ...
✅ Zip integrity OK: global-wheat-detection.zip

📦 Extracting global-wheat-detection.zip ... this may take a minute ...

🖼️ Images found in /content/global-wheat-detection/train: 3422

✅ Ready: dataset is prepared at /content/global-wheat-detection
   - CSV: True | path: /content/global-wheat-detection/train.csv
   - Train images: 3422


In [9]:
# STEP 3 — Parse bbox + 80/20 split (by image_id)

import ast, random, glob
from pathlib import Path
import pandas as pd
import numpy as np

SEED = 42
random.seed(SEED); np.random.seed(SEED)

DATA_DIR = Path("/content/global-wheat-detection")
CSV_PATH = DATA_DIR / "train.csv"
IMG_DIR  = DATA_DIR / "train"

# 1) Load CSV
df = pd.read_csv(CSV_PATH)
assert {'image_id','bbox','width','height'}.issubset(df.columns), "CSV missing required columns."

# 2) Keep only images that actually exist in IMG_DIR
existing = {Path(p).stem for p in glob.glob(str(IMG_DIR / "*.jpg"))}
df = df[df['image_id'].isin(existing)].copy()
assert len(df) > 0, "No matching rows after filtering to existing images."

# 3) Parse bbox column "[x, y, w, h]" to separate numeric columns
def parse_box(s):
    x, y, w, h = ast.literal_eval(s)  # safer than eval
    return x, y, w, h

xywh = df['bbox'].apply(parse_box)
df[['x','y','w','h']] = pd.DataFrame(xywh.tolist(), index=df.index)

# 4) Build unique image list & 80/20 split
image_ids = df['image_id'].drop_duplicates().tolist()
random.shuffle(image_ids)
split_idx = int(0.8 * len(image_ids))
train_ids = image_ids[:split_idx]
test_ids  = image_ids[split_idx:]

# 5) Save splits for later steps
SPLIT_DIR = Path("/content/wheat_split")
SPLIT_DIR.mkdir(parents=True, exist_ok=True)
(Path(SPLIT_DIR / "train_ids.txt")).write_text("\n".join(train_ids))
(Path(SPLIT_DIR / "test_ids.txt")).write_text("\n".join(test_ids))

# 6) Quick summary
n_imgs = len(image_ids)
n_tr, n_te = len(train_ids), len(test_ids)
n_boxes_tr = df[df['image_id'].isin(train_ids)].shape[0]
n_boxes_te = df[df['image_id'].isin(test_ids)].shape[0]

print("✅ Split complete")
print(f"Total unique images: {n_imgs}")
print(f"Train images: {n_tr}  | Annotations: {n_boxes_tr}")
print(f"Test  images: {n_te}  | Annotations: {n_boxes_te}")
print("\nSaved:")
print(f"- /content/wheat_split/train_ids.txt")
print(f"- /content/wheat_split/test_ids.txt")

# (We keep the parsed df in memory for exploration if you want:)
df.head()


✅ Split complete
Total unique images: 3373
Train images: 2698  | Annotations: 118152
Test  images: 675  | Annotations: 29641

Saved:
- /content/wheat_split/train_ids.txt
- /content/wheat_split/test_ids.txt


,image_id,width,height,bbox,source,x,y,w,h
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0,124.0,117.0


2.	YOLOv8 Training & Inference

o	Use the ultralytics library.

o	Train YOLOv8 on the wheat dataset for at least 20 epochs.

o	Evaluate using mAP (mean Average Precision) and inference speed (FPS).


In [2]:
# Install Ultralytics YOLOv8 library
!pip install ultralytics==8.3.0

# Verify install
import ultralytics
print("Ultralytics version:", ultralytics.__version__)


Ultralytics version: 8.3.0


YOLO label conversion (class = 0)

YOLO wants normalized (cx, cy, w, h) in txt per image.

In [ ]:
import cv2, os, tqdm

IM_DIR = "/content/global-wheat-detection/train"  # folder with images (jpg)
YOLO_IMG_DIR = f"{DATA}/images"
YOLO_LBL_DIR = f"{DATA}/labels"
for split in ["train","val"]:
    os.makedirs(f"{YOLO_IMG_DIR}/{split}", exist_ok=True)
    os.makedirs(f"{YOLO_LBL_DIR}/{split}", exist_ok=True)

val_ids = set(test_ids)  # use test as validation if you don't have a separate val set

grouped = df.groupby("image_id")
for image_id, g in tqdm.tqdm(grouped):
    img_path = f"{IM_DIR}/{image_id}.jpg"
    if not os.path.exists(img_path):
        continue
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    # YOLO lines
    lines = []
    for _, r in g.iterrows():
        x,y,bw,bh = r['x'], r['y'], r['w'], r['h']
        cx = (x + bw/2) / w
        cy = (y + bh/2) / h
        nw = bw / w
        nh = bh / h
        lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    split = "val" if image_id in val_ids else "train"
    cv2.imwrite(f"{YOLO_IMG_DIR}/{split}/{image_id}.jpg", img)
    with open(f"{YOLO_LBL_DIR}/{split}/{image_id}.txt","w") as f:
        f.write("\n".join(lines))


YOLO data.yaml

In [ ]:
yaml_text = f"""
path: {DATA}
train: images/train
val: images/val
names:
  0: wheat
"""
open(f"{DATA}/wheat.yaml","w").write(yaml_text)
print(open(f"{DATA}/wheat.yaml").read())


2) YOLOv8 training & inference (Ultralytics)

In [ ]:
!pip -q install ultralytics>=8.2.0 torchmetrics
from ultralytics import YOLO

yolo_model = YOLO('yolov8s.pt')  # small, faster
results = yolo_model.train(
    data=f"{DATA}/wheat.yaml",
    epochs=25,                 # ≥20 as required
    imgsz=1024,                # Global Wheat is high-res; 1024 is a good compromise
    batch=8,
    device=0,
    patience=5
)


YOLO evaluation & speed

In [ ]:
# mAP reported by Ultralytics val automatically (check results.csv or print below)
metrics = yolo_model.val()
print(metrics)  # includes mAP50 and mAP50-95

# Inference speed on ~100 images
import glob, time
test_imgs = sorted(glob.glob(f"{YOLO_IMG_DIR}/val/*.jpg"))[:100]
t0 = time.time()
_ = yolo_model(test_imgs, stream=False, verbose=False)  # warmup included
t1 = time.time()
ms_per_img = (t1 - t0) * 1000.0 / len(test_imgs)
print(f"YOLOv8s: {ms_per_img:.2f} ms/image  (~{1000/ms_per_img:.2f} FPS)")


Save qualitative outputs

In [ ]:
out_dir = "/content/yolo_vis"
os.makedirs(out_dir, exist_ok=True)
for p in test_imgs[:8]:
    res = yolo_model.predict(p, save=True, project=out_dir, name="preds", exist_ok=True, verbose=False)
print("Saved YOLO imgs under:", out_dir)


3) Faster R-CNN training & inference (torchvision)

Dataset for Faster R-CNN

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class WheatDetDataset(Dataset):
    def __init__(self, df, ids, img_dir, transforms=None):
        self.df = df
        self.ids = ids
        self.img_dir = img_dir
        self.transforms = transforms
        self.gdf = df.groupby("image_id")

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        image_id = self.ids[idx]
        img = Image.open(f"{self.img_dir}/{image_id}.jpg").convert("RGB")
        g = self.gdf.get_group(image_id)
        boxes = []
        for _, r in g.iterrows():
            x,y,w,h = r['x'], r['y'], r['w'], r['h']
            boxes.append([x, y, x+w, y+h])
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.ones((len(boxes),), dtype=torch.int64),  # class 1
            "image_id": torch.tensor([idx]),
        }
        if self.transforms: img = self.transforms(img)
        return img, target

import torchvision.transforms as T
train_ds = WheatDetDataset(df, [i for i in train_ids if (f"{IM_DIR}/{i}.jpg")], IM_DIR, T.ToTensor())
val_ds   = WheatDetDataset(df, [i for i in test_ids  if (f"{IM_DIR}/{i}.jpg")], IM_DIR, T.ToTensor())

def collate_fn(batch): return tuple(zip(*batch))
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=2)


Model, optimizer, training loop (~10–15 epochs)

In [ ]:
import torchvision
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
# set num_classes = 2 (background + wheat)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_sched = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

def train_one_epoch(model, loader):
    model.train()
    loss_sum = 0.0
    for imgs, targets in loader:
        imgs = [im.to(device) for im in imgs]
        targs = [{k: v.to(device) for k,v in t.items()} for t in targets]
        loss_dict = model(imgs, targs)
        loss = sum(loss_dict.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    return loss_sum / len(loader)

EPOCHS = 12
for ep in range(1, EPOCHS+1):
    tr_loss = train_one_epoch(model, train_loader)
    lr_sched.step()
    print(f"Epoch {ep}: loss {tr_loss:.4f}")


mAP with TorchMetrics

In [ ]:
!pip -q install torchmetrics>=1.3.0
from torchmetrics.detection.mean_ap import MeanAveragePrecision
metric = MeanAveragePrecision(iou_type="bbox")  # computes mAP50 and mAP50-95

model.eval()
with torch.no_grad():
    for imgs, targets in val_loader:
        imgs = [im.to(device) for im in imgs]
        preds = model(imgs)
        # convert to expected dicts on CPU
        preds_cpu = []
        for p in preds:
            preds_cpu.append({
                "boxes": p["boxes"].detach().cpu(),
                "scores": p["scores"].detach().cpu(),
                "labels": p["labels"].detach().cpu(),
            })
        targs_cpu = [{k: v.detach().cpu() for k,v in t.items()} for t in targets]
        metric.update(preds_cpu, targs_cpu)

metrics_frcnn = metric.compute()
print(metrics_frcnn)
# Access like: metrics_frcnn['map_50'], metrics_frcnn['map']


Inference speed

In [ ]:
import time, itertools

def iter_images(ids, limit=100):
    for i in ids[:limit]:
        yield Image.open(f"{IM_DIR}/{i}.jpg").convert("RGB")

imgs = [T.ToTensor()(im).to(device) for im in iter_images(test_ids, 100)]
torch.cuda.synchronize()
t0 = time.time()
with torch.no_grad():
    # single-image predictions for fair ms/image
    for im in imgs:
        _ = model([im])
torch.cuda.synchronize()
t1 = time.time()
ms_per_img = (t1 - t0) * 1000.0 / len(imgs)
print(f"Faster R-CNN: {ms_per_img:.2f} ms/image (~{1000/ms_per_img:.2f} FPS)")


Qualitative images

In [ ]:
import numpy as np, cv2
vis_dir = "/content/frcnn_vis"; os.makedirs(vis_dir, exist_ok=True)
model.eval()
count = 0
with torch.no_grad():
    for image_id in test_ids[:8]:
        img = cv2.imread(f"{IM_DIR}/{image_id}.jpg")
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        im_t = T.ToTensor()(Image.fromarray(rgb)).to(device)
        pred = model([im_t])[0]
        boxes = pred['boxes'].detach().cpu().numpy()
        scores= pred['scores'].detach().cpu().numpy()
        for (x1,y1,x2,y2), s in zip(boxes, scores):
            if s<0.4: continue
            cv2.rectangle(img,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),2)
            cv2.putText(img, f"{s:.2f}", (int(x1), max(0,int(y1)-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        cv2.imwrite(f"{vis_dir}/{image_id}.jpg", img)
print("Saved FRCNN imgs under:", vis_dir)


4) Results table (copy-paste and fill)

| Model           | Epochs | mAP\@0.5 | mAP\@0.5:0.95 | ms/image (↓) | Notes (good/missed) |
| --------------- | -----: | -------: | ------------: | -----------: | ------------------- |
| YOLOv8s         |     25 |          |               |              |                     |
| Faster R-CNN-50 |     12 |          |               |              |                     |


Interpretation tips

Expect YOLOv8 to be faster and often competitive or better at mAP50.

Faster R-CNN may do better on mAP50–95 if trained long enough and tuned (anchors, LR, aug).

Wheat heads are small/high-density; imgsz and augmentations impact results a lot.

5.	Report Preparation

o	Summarize findings in a short report.

o	Include metrics, sample output images, and a discussion on which model worked          better and why.  


1) Objective & Dataset (≈¼ page)

Goal. Compare YOLOv8s (one-stage) vs Faster R-CNN ResNet50-FPN (two-stage) on Global Wheat Detection in accuracy, speed, and robustness.
Dataset. Global Wheat Detection (Kaggle): 1 class (wheat head), ~3,300 images with bounding boxes [x, y, w, h].
Split. Train [80%], Val/Test [20%] (image-wise).
Preprocessing/labels. YOLO: normalized (cx, cy, w, h); Faster R-CNN: [x1, y1, x2, y2]. Image size used: [imgsz].

2) Methods (≈½ page)

YOLOv8s (Ultralytics).

Hyperparams: epochs [≥20, used: XX], imgsz [1024], batch [8], optimizer/aug: defaults.

Training command/log reference: [path/to/runs/train/…].

Faster R-CNN (torchvision).

Hyperparams: epochs [10–15, used: XX], batch [4], LR schedule [0.005 → StepLR(5, 0.1)].

Implementation: torchvision.models.detection.fasterrcnn_resnet50_fpn, predictor set to 2 classes (bg + wheat).

Evaluation protocol (same split & hardware).

Metrics: mAP@0.5, mAP@0.5:0.95.

Speed: ms/image (report single-image inference; warm-up + torch.cuda.synchronize()).

3) Results (Quantitative) (≈½ page)

| Model           | Epochs | mAP\@0.5 | mAP\@0.5:0.95 | ms/image (↓) | Notes                          |
| --------------- | -----: | -------: | ------------: | -----------: | ------------------------------ |
| YOLOv8s         |   \[ ] |     \[ ] |          \[ ] |         \[ ] | \[e.g., strong on clear heads] |
| Faster R-CNN-50 |   \[ ] |     \[ ] |          \[ ] |         \[ ] | \[e.g., tighter boxes?]        |


4) Results (Qualitative) (≈½ page)

Good detections (insert 2–3 images per model):

YOLOv8: [image + one-line caption].

Faster R-CNN: [image + one-line caption].

Missed/Failure cases (tiny heads, overlaps, occlusion, background FPs):

YOLOv8: [image + one-line caption].

Faster R-CNN: [image + one-line caption].

5) Discussion & Conclusion (≈½ page)

Which performed better and why.

Accuracy: At mAP@0.5, [winner] by [Δ]. At mAP@0.5:0.95, [winner] by [Δ].

Speed: [model] faster by [×] ( [ms/img] vs [ms/img] ).

Interpretation: One-stage YOLOv8 is typically faster and competitive at mAP@0.5; two-stage Faster R-CNN can improve localization precision (benefits 0.5:0.95) given enough training/tuning.

Robustness: [who handled tiny/overlapping heads better? Any systematic FPs?]

What to try next.

Higher imgsz (1280) for small objects; longer training ( +10–20 epochs ).

Stronger backbones (YOLOv8m/l, ResNet101-FPN).

Augmentations for density: Mosaic, RandomCrop, Copy-Paste.

Post-processing sweeps: conf/NMS thresholds.